<a href="https://colab.research.google.com/github/onism/MyLearning/blob/master/IMDB_Sent_Ana_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf 
import tensorflow_datasets as tfds 
import numpy as np

In [2]:
tf.config.list_physical_devices('GPU')

[]

In [3]:
# using TFDS dataset 

imdb_train, ds_info = tfds.load(name='imdb_reviews', split='train', with_info=True, as_supervised=True)

imdb_test = tfds.load(name='imdb_reviews', split='test', as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWPSJGT/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWPSJGT/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteWPSJGT/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [4]:
ds_info

tfds.core.DatasetInfo(
    name='imdb_reviews',
    version=1.0.0,
    description='Large Movie Review Dataset.
This is a dataset for binary sentiment classification containing substantially more data than previous benchmark datasets. We provide a set of 25,000 highly polar movie reviews for training, and 25,000 for testing. There is additional unlabeled data for use as well.',
    homepage='http://ai.stanford.edu/~amaas/data/sentiment/',
    features=FeaturesDict({
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=2),
        'text': Text(shape=(), dtype=tf.string),
    }),
    total_num_examples=100000,
    splits={
        'test': 25000,
        'train': 25000,
        'unsupervised': 50000,
    },
    supervised_keys=('text', 'label'),
    citation="""@InProceedings{maas-EtAl:2011:ACL-HLT2011,
      author    = {Maas, Andrew L.  and  Daly, Raymond E.  and  Pham, Peter T.  and  Huang, Dan  and  Ng, Andrew Y.  and  Potts, Christopher},
      title     = {Learning Word

In [5]:
for example, label in imdb_train.take(1):
    print(example, '\n', label)
    

tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string) 
 tf.Tensor(0, shape=(), dtype=int64)


In [6]:
tokenizer = tfds.features.text.Tokenizer()
vocabulary_set = set() 

MAX_TOKENS = 0 
for example, label in imdb_train:
    some_tokens = tokenizer.tokenize( example.numpy() )
    if MAX_TOKENS < len(some_tokens):
        MAX_TOKENS = len(some_tokens)
    vocabulary_set.update(some_tokens)

In [7]:
imdb_encoder = tfds.features.text.TokenTextEncoder(vocabulary_set, tokenizer=tokenizer)
vocab_size = imdb_encoder.vocab_size

print(vocab_size, MAX_TOKENS)

93931 2525


In [8]:
from tensorflow.keras.preprocessing import sequence 

def encode_pad_transform(sample):
    encoded = imdb_encoder.encode(sample.numpy())
    pad = sequence.pad_sequences([encoded], padding='post', maxlen=150)
    return np.array(pad[0], dtype=np.int64)

def encode_tf_fn(sample, label):
    encoded = tf.py_function(encode_pad_transform, inp=[sample], Tout=(tf.int64))
    encoded.set_shape([None])
    label.set_shape([])
    return encoded, label 

In [9]:
subset = imdb_train.take(10)
st = subset.map(encode_tf_fn)

In [10]:
for review, label in st.take(1):
    print(review, label)
    print(imdb_encoder.decode(review))

tf.Tensor(
[76217 86568 29084  8746 50049 75940 81185 47237 84829 81158 33013 19758
 56250 50337 77113 85831 67187 41717 74672 78742 51732 32551 72435 60824
 45148 84829  2745 80570 55169 33013 84592 13893  2745 78742 28901  7281
 78837 41168 72435 75940 78996 93637 93358 76217 75940 92153 29084 52708
 16732 69261 19425 17447 52490 49144 23399  5878 88896  5879 53637 71705
 57307 27270 88896 37445  2745 13166 90970  9543 63409 55932 27772 49823
 61320  4536 81159 33692  2411 93058 90301 50337 86568 69238 32551 29466
 23399 49456 74539 33013 29466 75940 55007 86568 64220 88629 30525  2928
 41094 47188 91998 54891 55007 63316 74672 27638 68206 72435  7821 17393
 78996 68206 56250 50337 78996 83321 74963 47188  7281  6978 65045 27975
 85575     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0], shape=(150,), dtype=int64) tf.Tensor(0, shape=(), dtype=int64)
This was an

In [11]:
encode_train = imdb_train.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)
encode_test = imdb_test.map(encode_tf_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [14]:
BATCH_SIZE = 128
vocab_size = imdb_encoder.vocab_size 
embedding_dim = 64 
rnn_units = 64

def build_lstm_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, mask_zero=True, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units),
        tf.keras.layers.Dense(1, activation='sigmoid')]
    )
    return model 

In [15]:
model = build_lstm_model( vocab_size, embedding_dim, rnn_units, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (128, None, 64)           6011584   
_________________________________________________________________
lstm_1 (LSTM)                (128, 64)                 33024     
_________________________________________________________________
dense_1 (Dense)              (128, 1)                  65        
Total params: 6,044,673
Trainable params: 6,044,673
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 'Precision', 'Recall'])
encode_train_batched = encode_train.batch(BATCH_SIZE).prefetch(90)
model.fit(encode_train_batched, epochs=10)

Epoch 1/10
195/196 [============================>.] - ETA: 0s - loss: 0.1819 - accuracy: 0.9347 - precision: 0.9354 - recall: 0.9340

InvalidArgumentError: ignored